__Unit Test__

The purpose of the notebook is to show the safety features in the data_store library that will accomodate all corner cases. 
Let's start with the imports:

In [12]:
from data_store import data_store
import time

- Creating an object for data_store. A database named 'database.json' has been created for me in my working directory as I have not provided a target directory.  

In [32]:
dataparser = data_store()

Database created in directory, C:\Users\Lenovo\Documents\fw_data_store/database.json


- Values are created with the format , obj.Create(key,value,Time-to-Live). 
- If no 'Time-to-live'(TTL) is provided, the value defaults to None.

In [33]:
dataparser.Create('Dwight',100,5)
dataparser.Create('Jim',500)
dataparser.Create('Michael',1000)

Value added
Value added
Value added


In [34]:
dataparser.Read('Jim')

'500'

- Read() returns the value of a valid key in the database. Let's see how it returns values with a TTL.

In [35]:
print('Before the wait')
val = dataparser.Read('Dwight')
print(val)
time.sleep(5)
print('After the wait')
val = dataparser.Read('Dwight')


Before the wait
100
After the wait


Exception: Key's Time-To-Live has expired. Unable to read.

- Create() also prohibits overwriting existing keys

In [36]:
dataparser.Create('Jim','Jim two')

Exception: Key is already present.

- Create() also blocks keys that are over 32 characters long or are not strings

In [37]:
dataparser.Create('blah' * 100, 100)

Exception: Key size is capped at 32. Given key length is 400

- The Delete() function pops a key from the database if it's present. 

In [41]:
dataparser.Delete('Jim')

Exception: Key not found in database

In [42]:
dataparser.Read('Jim')

Exception: Key not found in database

- Let's try deleting the key again

In [43]:
dataparser.Delete('Jim')

Exception: Key not found in database

 - DisplayAll() prints the saved data in the form of a dictionary with key-value pairs

In [40]:
dataparser.DisplayAll()

{'Dwight': 100, 'Michael': 1000}


- ClearAll() clears the database (after gettting confirmation of course)

In [31]:
dataparser.ClearAll()

Are you sure you want to clear the database? (y/n) y


Data cleared


In [29]:
dataparser.DisplayAll()

Exception: Unable to display. Database is empty